In [4]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from textblob import TextBlob
import time

# Set up Chrome driver (replace 'your/path/to/chromedriver' with the correct path)
chrome_driver_path = r"C:\chrome driver\chromedriver-win32\chromedriver.exe"
service = Service(chrome_driver_path)
driver = webdriver.Chrome(ChromeDriverManager().install())

# Function to scrape Twitter
def search_twitter(query):
    driver.get("https://twitter.com/")
    time.sleep(3)
    
    # Enter search query
    search_box = driver.find_element(By.CSS_SELECTOR, "[data-testid='SearchBox_Search_Input']")
    search_box.send_keys(query)
    search_box.send_keys(Keys.RETURN)
    time.sleep(3)
    
    tweets = []
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        tweet_elements = driver.find_elements(By.CSS_SELECTOR, "article [data-testid='tweetText']")
        for tweet in tweet_elements:
            tweets.append(tweet.text)
        
        # Scroll down to load more tweets
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        
        # Break if no new content is loaded
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    return tweets

# Function to scrape Instagram by hashtags
def search_instagram(query):
    driver.get(f"https://www.instagram.com/explore/tags/{query}/")
    time.sleep(3)
    
    posts = []
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        post_elements = driver.find_elements(By.CSS_SELECTOR, "article div div div div a")
        for post in post_elements:
            post_url = post.get_attribute('href')
            if post_url not in posts:
                posts.append(post_url)
        
        # Scroll down to load more posts
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        
        # Break if no new content is loaded
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    return posts

# Function to extract insights from text using TextBlob
def analyze_text(posts):
    keywords = ['affordable finance', 'financial literacy', 'digital payments', 'financial inclusion', 'financial security', 'fraud prevention']
    relevant_posts = []

    for post in posts:
        blob = TextBlob(post)
        for keyword in keywords:
            if keyword in post.lower():
                relevant_posts.append(post)
                print(f"Found keyword '{keyword}' in post: {post}")
                break  # Stop checking other keywords for this post once one is found

    return relevant_posts

# Main function to scrape both platforms and analyze the data
if __name__ == "__main__":
    # Twitter search
    twitter_query = "MSME finance Africa Nigeria"
    print("Searching Twitter for relevant posts...")
    twitter_posts = search_twitter(twitter_query)
    print(f"Total tweets scraped: {len(twitter_posts)}")

    # Instagram search
    instagram_query = "msmefinance"
    print("Searching Instagram for relevant posts...")
    instagram_posts = search_instagram(instagram_query)
    print(f"Total Instagram posts scraped: {len(instagram_posts)}")

    # Combine posts for analysis
    combined_posts = twitter_posts + instagram_posts

    # Analyze text for financial issues
    print("Analyzing posts for relevant financial topics...")
    relevant_posts = analyze_text(combined_posts)

    print(f"Total relevant posts found: {len(relevant_posts)}")

    # Close the driver after scraping
    driver.quit()

AttributeError: 'str' object has no attribute 'capabilities'